This notebook is for doing SanityCheck of the FLAT dataflow. All data in this file are randomly generated. \\
Author: Zeyu Chen


In [ ]:
# Import library
import tensorflow as tf
import numpy as np

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Unit Test Test Data
"""
# Set a random query_input with size 32*32*2*10 Batch_size * source_length * num_head * dim_per_head
query_input = np.ones([64, 64, 16, 64])
temp = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])[None, None, :, None]
query_input= query_input * temp
query_input
# Create a random key_input with size 32 * 64 * 2 * 10 Batch_size * query_length * num_head * dim_per_head
key_input = np.ones([64, 64, 16, 64])
temp = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160])[None, None, :, None]
key_input= key_input * temp
key_input
# Create random bias
bias = np.random.random([64, 16, 64, 64])
bias.shape
# Create random value matrix with size 32 * 64 * 2 * 10 Batch_size * length * head_num * dim_per_num
values = np.random.randint(0, 100, [64, 64, 16, 64])

In [ ]:
# Create logit output
logits = tf.einsum("BTNH,BFNH->BNFT", key_input, query_input)
# Add logits with bias
logits += bias[0, 0, :, :]
# Pass through the softmax layer
weights = tf.nn.softmax(logits, name="attention_weights")
# Pass through the dropout
#weights = tf.nn.dropout(weights, rate=0.4)
# Original code attention output
old_attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, values)
old_attention_output.shape

TensorShape([64, 64, 16, 64])

In [ ]:
# Prepare data for testing
query = tf.convert_to_tensor(query_input)
key = tf.convert_to_tensor(key_input)
bias = tf.convert_to_tensor(bias)
values = tf.convert_to_tensor(values)


# FLAT Implementation   ---  First Version
#@title First Implement Version
"""Fuse Logit and Attend operators

Args:
  query_input: query output from query_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
  key_input: key output from key_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
  value_input: value output from value_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
  bias_matrix: A tensor with shape [batch_size, 1, length_query, length_source],

Symbol meanings:
  B : batch size
  T : key_length == length_qeury
  N : Head_num
  H : dim_per_head
  F : query_length == length_source (which also commented as )
Returns:
  Attention layer output with shape [batch_size, length_query, hidden_size]
  logits = tf.einsum("BTNH,BFNH->BNFT", key, query)
  attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, value)
"""
# Randomly set granularity now
batch_granularity = 1
head_granularity = 1
element_granularity = 1

# Get shape of the matrix
batch_size, source_length, head_num, dim = query.shape.as_list()
_,key_length,_,_ = key.shape.as_list()
output = None
assert (head_granularity < head_num)

# The outermost loop loops through batch_size, with granularity as stride
for batch in range(0, batch_size, batch_granularity):
  flag = True
  suboutput = None
  # The inner loop loops through head, with granularity as stride
  for head in range(0, head_num, head_granularity):
    #Head_Dimension = len(tf.unstack(query, axis = 3))
    #for element in range(0, Head_Dimension, element_granularity):
      #for loop in range(1):
    output_from_unit_batch = np.zeros([batch_granularity, source_length, head_granularity, dim])
    # Check each bach inside each granularity
    for unit_batch in range(batch, batch + batch_granularity):
      output_from_unit_head = np.zeros([source_length, head_granularity, dim])
      # Check each head in the head granularity
      for unit_head in range(head, head + head_granularity):
        #for unit_element in range(element):
        logit_output = np.ones([source_length, key_length])
        for loop in range(2):
          if loop == 0:
            #iterate through query_length
            for unit_source in range(source_length):
              query_source = query_input[unit_batch, unit_source, unit_head, :]
              query_source = tf.reshape(query_source, [1, query_source.size]) # Tensor with 1 * 10 (unit_source_length * dim_per_head)
              key_source = key[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (key length * dim_per_head)
              #Result is F(1) * T matrix
              #Not sure if we need to do transpose here
              #result = tf.matmul(query_source, key_source) # Tensor with 1 * 64 size (unit_source_length * key_length)
              result = tf.matmul(query_source, tf.transpose(key_source)) # Tensor with 1 * 64 size (unit_source_length * key_length)
              result += bias[unit_batch, :, unit_source, :]
              #Row granularity
              result = tf.nn.softmax(result, name="attention_weights", axis=-1)
              result = tf.nn.dropout(result, rate=0.4)
              logit_output[unit_source, :] = result
            #Logit output now is F * T matrix
            logit_output = tf.convert_to_tensor(logit_output)
          else:
            value_source = values[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (T * H) query_length * dim_per_head
            logit_output = tf.cast(logit_output, dtype=float)
            value_source = tf.cast(value_source, dtype=float)
            attention_output = tf.matmul(logit_output, value_source) #Matrix with size F * H
        output_from_unit_head[:, unit_head - head, :] = attention_output
      #Size now is F*N*H (output_from_unit_head)
      output_from_unit_batch[unit_batch - batch, :, :, :] = output_from_unit_head
    # Size now is BFNH (output from unit batch)
    if flag == True:
      suboutput = output_from_unit_batch
      flag = False
    else:
      suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
  if output == None:
    output = suboutput
  else:
    output = tf.concat([output, suboutput], 0)
print(output.shape)


In [ ]:
# Optimized Version
# 加入dropout层后，结果便无法比较
batch_size, source_length, head_num, dim = tf.shape(query_input).numpy()
_,key_length,_,_, = tf.shape(key_input).numpy()
batch_granularity = 32
head_granularity = 8
# Broadcasting to reshape the bias matrix to the shape as the logit BNFT
# The outermost loop loops through batch_size, with granularity as stride
for batch in tf.range(0, batch_size, batch_granularity):
  for head in tf.range(0, head_num, head_granularity):
    # Next Line removed now since input size is fixed and the result is evenly divided
    batch_termination = batch + batch_granularity if batch + batch_granularity <= batch_size else batch_size
    for unit_batch in tf.range(batch, batch_termination):
      # Next Line removed now since input size is fixed and the result is evenly divided
      head_termination = head + head_granularity if head + head_granularity <= head_num else head_num
      for unit_head in tf.range(head, head + head_granularity):
        for loop in tf.range(2):
          if loop == 0:
            # Tensorflow not support multiple indexing
            for query_source in tf.split(query_input[unit_batch, :, unit_head, :], num_or_size_splits=1, axis=0):
              key_source = key_input[unit_batch, :, unit_head, :]
              result = tf.matmul(query_source, tf.transpose(key_source))
              result += bias[unit_batch, unit_head, :, :]
              #Row granularity
              result = tf.nn.softmax(result, name="attention_weights")
              #result = tf.nn.dropout(result, rate=0.4)
              #logit_output = result
          else:
            value_source = values[unit_batch, :, unit_head, :] 
            attention_output = tf.matmul(result, value_source) #Matrix with size F * H
        if unit_head == head:
          attention_output = tf.expand_dims(attention_output, axis=1)
          output_from_unit_head = attention_output
        else:
          attention_output = tf.expand_dims(attention_output, axis=1)
          output_from_unit_head = tf.concat([output_from_unit_head, attention_output], 1)
      if unit_batch == batch:
        output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
        output_from_unit_batch = output_from_unit_head
      else:
        output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
        output_from_unit_batch = tf.concat([output_from_unit_batch, output_from_unit_head], 0)
    if head == 0:
      suboutput = output_from_unit_batch
    else:
      suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
  if batch == 0:
    output = suboutput
  else:
    output = tf.concat([output, suboutput], 0)
output.shape

TensorShape([64, 64, 16, 64])

In [ ]:
old_attention_output = tf.cast(old_attention_output, tf.float64)
np.all(np.abs(old_attention_output - output) < 1)

True

In [ ]:
# What if user wants to try even higher granularity?
# new_res = []
# if (tf.config.list_physical_devices('GPU')):
#   tf.keras.backend.clear_session()
#   print("Before doing attention operations. Reset the memory")
#   tf.config.experimental.reset_memory_stats('GPU:0')
#   memory1 = tf.config.experimental.get_memory_info('GPU:0')
#   print(memory1['peak'])
#   print(memory1['current'])
#   f.write("Before running : (%f, %f)\n" % (memory1['peak'], memory1['current']))
#   for i in range(matrix_num):
# qeury = query_matrix[i]
# key = key_matrix[i]
# value = value_matrix[i]
batch_size, source_length, head_num, dim = tf.shape(query_input).numpy()
_,key_length,_,_, = tf.shape(key_input).numpy()
batch_granularity = 32
head_granularity = 16
bias_value = bias[0, 0, :, :]
for batch in tf.range(0, batch_size, batch_granularity):
  for head in tf.range(0, head_num, head_granularity):
    for unit_batch in tf.range(batch, batch + batch_granularity):
      for unit_head in tf.range(head, head + head_granularity, 8):
        # query_source now should be 1 * 64 * 8 * 64
        query_source = tf.gather(query_input[unit_batch, :, :, :], indices=tf.range(head, head + 8), axis=1)
        key_source = tf.gather(key_input[unit_batch, :, :, :], indices=tf.range(head, head + 8), axis=1)
        result = tf.einsum("TNH, FNH->NFT", key_source, query_source)
        result += bias_value[None, :, :]
        result = tf.nn.softmax(result, name="attention_weights")
        #result = tf.nn.dropout(result, rate=0.4)
        if unit_head == head:
          logit = result
        else:
          # Concatenate over head num dimension
          logit = tf.concat([logit, result], axis=0)
      value_source = values[unit_batch, :, :, :]
      attention_output = tf.einsum("NFT,TNH->FNH", logit, value_source)
      if unit_batch == batch:
        attention_output = tf.expand_dims(attention_output, axis=0)
        output_from_unit_batch = attention_output
      else:
        attention_output = tf.expand_dims(attention_output, axis=0)
        output_from_unit_batch = tf.concat([output_from_unit_batch, attention_output], 0)
    if head == 0:
      suboutput = output_from_unit_batch
    else:
      suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
  if batch == 0:
    output = suboutput
  else:
    output = tf.concat([output, suboutput], 0)
    # new_res.append(output)
    # memory2 = tf.config.experimental.get_memory_info('GPU:0')
    # print("Memory profiling using FLAT")
    # print(memory2['peak'])
    # print(memory2['current'])
    # f.write("After Iteration %d : (%f, %f)\n" % (i, memory2['peak'], memory2['current']))
# f.close()

In [ ]:
old_attention_output = tf.cast(old_attention_output, tf.float64)
np.all(np.abs(old_attention_output - output) < 1)

True